# myhmmlearn
刘建平Blog: https://www.cnblogs.com/pinard/p/6955871.html

In [47]:
# -*- coding: utf-8 -*-
# Created on Mon Sep 30 09:59:48 2019

import numpy as np
'''
self.transmits = np.array([[0.5, 0.2, 0.3],
                               [0.3, 0.5, 0.2],
                               [0.2, 0.3, 0.5]])      # transmits[i][j]: 从状态i转移到j的概率
        
self.emits = np.array([ [0.5, 0.4, 0.7],
                                [0.5, 0.6, 0.3] ])            # emits[i][j]: 观测i被隐状态j发射的概率
        
self.start_prob = np.array([0.2, 0.4, 0.4])
'''
class MultinomialHMM:
    def __init__(self):
        self.n_states = None
        self.n_iter = None
        self.tol = None
    
        self.transmits = np.array([[0.5, 0.2, 0.3],
                               [0.3, 0.5, 0.2],
                               [0.2, 0.3, 0.5]])      # transmits[i][j]: 从状态i转移到j的概率
        
        self.emits = np.array([ [0.5, 0.4, 0.7],
                                [0.5, 0.6, 0.3] ])            # emits[i][j]: 观测i被隐状态j发射的概率
        
        self.start_prob = np.array([0.2, 0.4, 0.4]) 
    
    # enable一次输入多个观测序列来学习
    def fit(self, observations, n_iter = 5):     # 根据观测值用鲍姆韦尔奇算法学习模型参数, 转移发射起始
        if not self.n_states:
            raise Exception('请给出可能的隐状态数')
            
        self.n_observation = self.get_num_observation(observations)   # 获取不同观察值的数目
        # 要有每一时间步的各个状态的前向, 后向概率, 分为前向概率数组和后向概率数组
        for iter in range(0, n_iter):
            forward_scores = self.decode(observations, True)   # 获取前向概率
            backward_scores = self.get_backward_scores(observations)   # 获取后向概率
            forward_mul_backward_scores = forward_scores * backward_scores   # 对应相乘
            
            self.start_prob = forward_mul_backward_scores[0] / np.sum(forward_mul_backward_scores[0])

            # 学习转移矩阵, 由于考虑的是转移概率, 最后一个时间步不用再转移, 所以不考虑最后一个时间步
            trans_fenmu = forward_mul_backward_scores[:-1] / np.sum(forward_mul_backward_scores[:-1], 1).reshape(-1, 1)
            trans_fenmu = np.sum(trans_fenmu, 0)
            for i in range(0, self.n_states):
                temp = forward_scores[ :-1, i]   # Alpha
                fenzi = np.matmul(temp.reshape(temp.shape[0], 1), self.transmits[i].reshape(1,-1))  # * Alpha(i,j)
                fenzi *= self.emits[observations[1:]]   # * beta(Ot+1), 发射概率
                fenzi *= backward_scores[1:]            # * Beta(t+1), 下一时间步的后向概率
                fenzi = np.sum(fenzi, 0)

                self.transmits[i] = fenzi / trans_fenmu[i]
            print(self.transmits)
#             print(forward_scores)

            # 学习发射矩阵, 每一个时间步都要考虑发射概率
            emit_fenmu = forward_mul_backward_scores / np.sum(forward_mul_backward_scores, 1).reshape(-1, 1)
            emit_fenmu = np.sum(emit_fenmu, 0)
            for i in range(0, self.n_observation):
                temp = np.where(observations == i)         # Ot = Vk
                temp = forward_mul_backward_scores[temp] / np.sum(temp, 1)
                self.emits[i] = np.sum(temp, 0) / emit_fenmu[i]
        
#             print(self.transmits)
        
    def get_num_observation(self, observations):
        return len(set(observations))
        
    def get_backward_scores(self, observations):
        backward_scores = [[1] * self.n_states]   # 最后1个时间步的各隐状态后向概率初始化为1
        
        for i in range(1, len(observations)):
            temp_score = []
            for j in range(0, self.n_states):
                temp_score.append(np.sum((np.array(backward_scores[-i]) * self.transmits[j] * self.emits[observations[-i]])))   
            backward_scores.append(temp_score)
            
        backward_scores.reverse()
        return np.array(backward_scores)
    
    def decode(self, observations, get_forward_scores = False):         # 根据观测得到最大概率隐状态序列及其分数
        all_route = []
        forward_scores = []      # 保存每一时间步的前向概率, 为学习模型提供前向概率
        forward_score = self.start_prob * self.emits[observations[0]]
        forward_scores.append(forward_score.tolist())
        for t in range(1, len(observations)):
            trans_score = forward_score * self.transmits.T
            all_route.append(np.argmax(trans_score, 1))
            forward_score = np.max(trans_score, 1) * self.emits[observations[t]]
            forward_scores.append(forward_score.tolist())
        
        if get_forward_scores == True:
            return np.array(forward_scores)       # 学习模型时调用decode()获取每一时间步的前向概率
        
        all_route.reverse()
        next_route = np.argmax(forward_score)     # 最后1个隐状态
        final_route = [next_route]                # 回溯路径
        for route in all_route:
            next_route = route[next_route]
            final_route.append(next_route)
        
        final_route.reverse()
        max_route_score = np.log(np.max(forward_score))
        return final_route, max_route_score
    
    def score(self, observations):          # 模型学习完成, 对观察序列作出评分
        forward_score = self.start_prob * self.emits[observations[0]]
        for i in range(1, len(observations)):
            forward_score = np.matmul(forward_score, self.transmits)   # 转移后的前向分数
            forward_score *= self.emits[observations[i]]    # 转移后各状态发射出当前观测的前向分数
        
        return np.log(np.sum(forward_score))    # 返回的分数是loge(score)

In [57]:
from hmmlearn import hmm
model = hmm.MultinomialHMM(n_components=3, verbose=True)
seen = np.array([[0,1,0,1]])
model.fit(seen)
model.transmat_

         1          -2.7727             +nan
         2          -2.6556          +0.1171
         3          -2.4414          +0.2142
         4          -1.8462          +0.5952
         5          -0.7338          +1.1124
         6          -0.0971          +0.6366
         7          -0.0027          +0.0944
         8          -0.0000          +0.0027


array([[1.90306598e-12, 1.00000000e+00, 1.55057596e-19],
       [3.34367059e-01, 2.38286491e-44, 6.65632941e-01],
       [5.67707496e-13, 1.00000000e+00, 1.29897287e-20]])